# Case Quod

## Geração de dados

In [1]:
import pandas as pd # Importa a biblioteca 'pandas', essencial para análise e manipulação de dados estruturados (DataFrames). É convencionalmente apelidada de 'pd'.
from datetime import datetime, timedelta # Importa as classes 'datetime' e 'timedelta' do módulo 'datetime'.
                                        # 'datetime' é usada para manipular datas e horas específicas.
                                        # 'timedelta' é usada para representar durações ou diferenças entre momentos.
import random as rd # Importa a biblioteca 'random', utilizada para gerar números e seleções pseudoaleatórias. É convencionalmente apelidada de 'rd'.

informações sobre a simulação de dados
- dataset de vendas
- mínimo de 50 registros
- colunas: ID, Data, Produto, Categoria, Quantidade,
Preço.


In [2]:
def data_aleatoria(inicio, fim): # Define a função 'data_aleatoria' que aceita dois parâmetros de data/hora: 'inicio' e 'fim'.
    # O objetivo da função é retornar uma data aleatória dentro do intervalo [inicio, fim].
    try: # Inicia um bloco 'try' para lidar com possíveis erros durante a execução, como datas inválidas ou ordem incorreta (fim < inicio).
        delta = fim - inicio # Variável 'delta': Calcula a diferença entre a data final e a data inicial. O resultado é um objeto 'timedelta'.

        # Variável 'dias_aleatorios': Gera um número inteiro aleatório ('randint')
        # entre 0 (início) e o número total de dias no 'delta' (intervalo).
        dias_aleatorios = rd.randint(0, delta.days) 

        # Retorna a data inicial ('inicio') adicionada ao número de dias aleatórios.
        # Usa 'timedelta' para garantir que a soma seja uma nova data válida.
        return inicio + timedelta(days=dias_aleatorios) 

    except Exception as e: # Captura qualquer exceção ('Exception') que possa ocorrer no bloco 'try'.
        # Bloco de tratamento de erro: Imprime a mensagem de erro original (e) seguida de uma mensagem de contexto.
        # Nota: Idealmente, em código de produção, você pode levantar a exceção ('raise e') ou logar o erro em vez de apenas imprimir.
        print(e, "Erro ao gerar data aleatória.", sep='/')

In [3]:
selecionar = lambda x: rd.choice([x, None]) # Função lambda 'selecionar': Recebe um argumento 'x' e retorna um valor aleatório de uma lista que contém 'x' e 'None'.

In [4]:
def gerar_dados(data_inicial: datetime, data_final: datetime, produtos: list, tamanho: int, semente:int) -> pd.DataFrame:
    # Define a função 'gerar_dados'.
    # Recebe: datas de início/fim (datetime), uma lista de 'produtos', o 'tamanho' do DataFrame (int)
    # e uma 'semente' para o gerador aleatório (int).
    # O '-> pd.DataFrame' indica que a função retornará um objeto pandas DataFrame (type hinting).

    lista = [] # Inicializa a lista vazia que irá armazenar os dicionários (futuras linhas do DataFrame).

    # Variável 'seed': Configura a semente ('semente') do gerador de números pseudoaleatórios.
    # Isto garante que a mesma sequência de dados aleatórios seja gerada a cada execução com a mesma 'semente'.
    rd.seed(semente)

    # Loop principal: Itera 'tamanho' vezes (para gerar o número desejado de linhas).
    for _ in range(tamanho):
        try: # Inicia um bloco 'try' para capturar erros durante a geração de cada linha.

            # Variável 'valor': Seleciona aleatoriamente um elemento da lista 'produtos'.
            # Assume-se que cada elemento em 'produtos' é uma tupla ou lista com múltiplos itens.
            valor = rd.choice(produtos)

            # Bloco de criação do registro (linha):
            lista.append({
                "id": selecionar(valor[0]), # Pega o primeiro elemento do 'valor' como 'id'.
                # Chama a função externa 'data_aleatoria' para gerar uma data entre 'data_inicial' e 'data_final'.
                "Data": selecionar(data_aleatoria(data_inicial, data_final)),
                "Produto": selecionar(valor[1]), # Pega o segundo elemento como 'Produto'.
                "Categoria": selecionar(valor[2]), # Pega o terceiro elemento como 'Categoria'.
                # Gera um inteiro aleatório para a 'Quantidade' entre 10 e 120.
                "Quantidade": selecionar(rd.randint(valor[3][0], valor[3][1])),
                # Gera um número de ponto flutuante ('float') para o 'Preço' entre 500 e 2000, arredondando para 2 casas decimais.
                "Preço": selecionar(round(rd.uniform(valor[4][0], valor[4][1]), 2))
            })

        except Exception as e: # Captura qualquer exceção genérica.
            # Imprime o erro e uma mensagem de contexto, separando-os por '/'.
            # A execução do loop continua se um erro for tratado aqui.
            print(e, "Erro ao gerar linha do dataframe.", sep='/')

    # Após o loop, converte a 'lista' de dicionários em um DataFrame do pandas e o retorna.
    return pd.DataFrame(lista)

A função ```gerar_dados``` cria um dataframe selecionando valores aleatórios usando a biblioteca random
a ideia princinpal é deixar que os métodos de aleatóriedade selecionem as linhas do dataframe.

Para gerar os dados deve-se passar o intervalo de datas, para que sejam criadas datas aleatórias. Continuando, deve-se passar uma lista de listas com id, 
o produto e sua categoria. Por fim, deve-se concluir com o tamanho do dataframe e uma semente geradora, para fixar sempre a mesma tabela.



In [5]:
produtos_pc = [
    [1,"Processador Intel Core i7 13ª Geração", "Processador",(1,3),(1500.00, 2500.00)],
    [2,"Placa-mãe ASUS Prime B660M", "Placa-mãe",(1,25),(800.00, 1400.00)],
    [3,"Memória RAM DDR5 16GB 5200MHz", "Memória RAM",(2,60),(300.00, 600.00)],
    [4,"SSD NVMe 1TB", "Armazenamento",(5,8),(350.00, 700.00)],
    [5,"Placa de vídeo NVIDIA RTX 4070 Ti", "Placa de vídeo",(1,15),(4000.00, 5500.00)],
    [6,"Fonte modular 750W 80 Plus Gold", "Fonte",(3,5),(400.00, 800.00)],
    [7,"Gabinete ATX com lateral em vidro", "Gabinete",(2,40),(250.00, 600.00)],
    [8,"Cooler a ar para CPU", "Cooler a ar",(5,7),(120.00, 300.00)],
    [9,"Water cooler 240mm", "Water cooler",(1,2),(350.00, 700.00)],
    [10,"Ventoinha RGB 120mm", "Ventoinha",(5,10),(50.00, 120.00)],
]

In [6]:
inicio = datetime(2023,1,1)
fim = datetime(2023,12,31)

dados = gerar_dados(inicio, fim, produtos_pc, 5000, 190)
dados

,id,Data,Produto,Categoria,Quantidade,Preço
0,2.0,NaT,Placa-mãe ASUS Prime B660M,Placa-mãe,NaN,1372.60
1,2.0,2023-10-28,None,None,13.0,NaN
2,NaN,2023-12-22,None,None,7.0,NaN
3,4.0,2023-07-12,None,None,NaN,673.41
4,5.0,2023-05-16,Placa de vídeo NVIDIA RTX 4070 Ti,None,NaN,4169.27
...,...,...,...,...,...,...
4995,NaN,2023-09-05,Placa-mãe ASUS Prime B660M,Placa-mãe,NaN,1246.99
4996,NaN,NaT,None,Fonte,5.0,471.13
4997,NaN,2023-02-15,None,None,2.0,NaN
4998,NaN,NaT,None,Memória RAM,35.0,NaN


## Tratamento de dados

Tratamento de dados, ou limpeza de dados, pode ser entendido como o ato de corrigir erros em dados. 
Por exemplo: remover duplicatas

- Verificação da Tipagem dos dados

In [7]:
def tabela_tipagem(dados: pd.DataFrame) -> pd.DataFrame:
    # A função 'tabela_tipagem' recebe um DataFrame do pandas como entrada e retorna outro DataFrame.
    # O objetivo é criar uma tabela que mostra a tipagem (tipo de dados) de cada coluna no DataFrame original.

    # Cria um novo DataFrame com duas colunas: 'Coluna' e 'Tipo de Dado'.
    # 'Coluna' contém os nomes das colunas do DataFrame original.
    # 'Tipo de Dado' contém o tipo de dado (dtype) correspondente a cada coluna.
    tabela = pd.DataFrame({
        "Coluna": dados.columns, # Obtém os nomes das colunas do DataFrame original.
        "Tipo de Dado": [dados[coluna].dtype for coluna in dados.columns] # Lista compreensiva que obtém o tipo de dado para cada coluna.
    })

    return tabela # Retorna o DataFrame resultante que mostra a tipagem das colunas.

In [8]:
tabela_tipagem(dados)

,Coluna,Tipo de Dado
0,id,float64
1,Data,datetime64[ns]
2,Produto,object
3,Categoria,object
4,Quantidade,float64
5,Preço,float64


Observamos que há duas colunas com tipagem erradas sendo elas id e quantidade. Ambas deveriam pertencer a classe de números inteiros.

In [9]:
# Realizando o método de mudança de tipagem
dados["id"] = pd.to_numeric(dados["id"], errors='coerce').astype('Int64') # Converte a coluna 'id' para numérico, tratando erros e convertendo para Int64.
dados["Quantidade"] = pd.to_numeric(dados["Quantidade"], errors='coerce').astype('Int64') # Converte a coluna 'Quantidade' para numérico, tratando erros e convertendo para Int64.

Verificando alteração

In [10]:
tabela_tipagem(dados)

,Coluna,Tipo de Dado
0,id,Int64
1,Data,datetime64[ns]
2,Produto,object
3,Categoria,object
4,Quantidade,Int64
5,Preço,float64


- Verificação de duplicatas e valores nulos

verificando se os dados gerados contém valores duplicados, ou seja linhas iguais.

In [11]:
dados.duplicated().sum()

np.int64(846)

Observamos que os possuem um total de 1488 de linhas duplicadas.

In [12]:
# Removendo os dados duplicados
dados = dados.drop_duplicates().reset_index(drop=True)

In [13]:
dados.duplicated().sum()

np.int64(0)

Atráves do método drop_duplicated observamos que as linhas duplicadas foram excluídas com sucesso.

In [14]:
# Criando função para verificar valores nulos
def verificar_nulos(dados: pd.DataFrame) -> pd.DataFrame:
    # A função 'verificar_nulos' recebe um DataFrame do pandas como entrada e retorna outro DataFrame.
    # O objetivo é criar uma tabela que mostra a quantidade de valores nulos em cada coluna do DataFrame original.

    # Cria um novo DataFrame com duas colunas: 'Coluna' e 'Valores Nulos'.
    # 'Coluna' contém os nomes das colunas do DataFrame original.
    # 'Valores Nulos' contém a contagem de valores nulos (NaN) para cada coluna.
    tabela_nulos = pd.DataFrame({
        "Coluna": dados.columns, # Obtém os nomes das colunas do DataFrame original.
        "Quantidade de Valores Nulos": [dados[coluna].isnull().sum() for coluna in dados.columns], # Lista compreensiva que conta os valores nulos para cada coluna.
        "Percentual de Valores Nulos": [round(dados[coluna].isnull().sum()/len(dados[coluna]) * 100,2) for coluna in dados.columns] # Lista compreensiva que calcula o percentual de valores nulos para cada coluna.

    })

    return tabela_nulos # Retorna o DataFrame resultante que mostra a contagem de valores nulos nas colunas.

In [15]:
verificar_nulos(dados)

,Coluna,Quantidade de Valores Nulos,Percentual de Valores Nulos
0,id,2068,49.78
1,Data,1689,40.66
2,Produto,2055,49.47
3,Categoria,2075,49.95
4,Quantidade,1972,47.47
5,Preço,1614,38.85


Observamos que os dados possuem uma extrema instência na falta de informações. Embora seja possível realizar tratamentos com imputações de dados
a grande quantidade de valores ausentes não torna isso o ideial no momento. Por isso, serão excluídos.

In [16]:
dados.dropna(inplace=True) # Remove linhas com valores nulos do DataFrame 'dados' diretamente (inplace=True).

In [17]:
dados.shape

(79, 6)

Por fim, ficaremos apenas com os dados sem nenhuma inconsoistência

In [18]:
dados.to_csv("dados_vendas.csv", index=False) # Salva o DataFrame 'dados' em um arquivo CSV chamado "dados_vendas.csv" sem incluir o índice das linhas (index=False).

Calculando o Total de vendas:

(quantidade * preço)

In [19]:
dados['Total de Vendas'] = dados['Quantidade'] * dados['Preço'] # Cria uma nova coluna 'Total de Vendas' no DataFrame 'dados', calculando o produto das colunas 'Quantidade' e 'Preço'. 
dados.head()

,id,Data,Produto,Categoria,Quantidade,Preço,Total de Vendas
104,2,2023-11-27,Placa-mãe ASUS Prime B660M,Placa-mãe,2,1263.57,2527.14
125,1,2023-03-22,Processador Intel Core i7 13ª Geração,Processador,1,1938.78,1938.78
202,4,2023-01-25,SSD NVMe 1TB,Armazenamento,5,444.64,2223.2
273,9,2023-10-17,Water cooler 240mm,Water cooler,2,440.26,880.52
347,1,2023-04-19,Processador Intel Core i7 13ª Geração,Processador,3,1657.05,4971.15


In [20]:
dados[dados['Total de Vendas'] == dados['Total de Vendas'].max()]

,id,Data,Produto,Categoria,Quantidade,Preço,Total de Vendas
3149,5,2023-05-11,Placa de vídeo NVIDIA RTX 4070 Ti,Placa de vídeo,12,5473.04,65676.48


In [21]:
import sqlite3

conexao = sqlite3.connect('dados_clean.db') # Cria uma conexão com o banco de dados SQLite chamado 'dados_vendas.db'. Se o arquivo não existir, ele será criado.
dados.to_sql('vendas', conexao, if_exists='replace', index=False) # Salva o DataFrame 'dados' em uma tabela SQL chamada 'vendas' no banco de dados conectado.

79